# This is a demo to get you more familiar with how to merge tabular data

In [1]:
import os
import sys
from functools import reduce
import pandas as pd
import numpy as np

from brickstudy import foundation

### now we will make some sample just data just as an example

In [3]:

# initialize list of lists
data1 = [['frank', 11], ['sushi', 15], ['julie', 14]]
 
# Create the pandas DataFrame
df1 = pd.DataFrame(data1, columns=['Name', 'Age'])
df1

,Name,Age
0,frank,11
1,sushi,15
2,julie,14


In [4]:
data2 = [['frank', 'Zaandam'], ['julie', 'Rotterdam'], ['sushi', 'Amstersdam']]
 
# Create the pandas DataFrame
df2 = pd.DataFrame(data2, columns=['Name', 'City'])
df2

,Name,City
0,frank,Zaandam
1,julie,Rotterdam
2,sushi,Amstersdam


# We just made two datasets. Now we will merge them. Just sticking them together randomly will not work. We want to use the names as a key. We mant pandas to reorganize the dataframes, based on that key, so all the right info goes with the right person. 

In [5]:
merged = df1.merge(df2, left_on='Name', right_on='Name')

In [6]:
merged

,Name,Age,City
0,frank,11,Zaandam
1,sushi,15,Amstersdam
2,julie,14,Rotterdam


## Ok that was easy. Let's move on to what you have...excel spreadsheets with keys named at random, which need to be matched (your computer is not psychic)

In [7]:
# Scenario A: different common key

# initialize list of lists
data3 = [['frank', 11], ['sushi', 15], ['julie', 14]]
 
# Create the pandas DataFrame
df3 = pd.DataFrame(data1, columns=['Not_the_same', 'Age'])
df3 

,Not_the_same,Age
0,frank,11
1,sushi,15
2,julie,14


In [8]:
data4 = [['frank', 'Zaandam'], ['julie', 'Rotterdam'], ['sushi', 'Amstersdam']]
 
# Create the pandas DataFrame
df4 = pd.DataFrame(data2, columns=['Key_is_notCommon', 'City'])
df4

,Key_is_notCommon,City
0,frank,Zaandam
1,julie,Rotterdam
2,sushi,Amstersdam


In [11]:
merged2 = df3.merge(df4, left_on='Not_the_same', right_on='Key_is_notCommon')

In [12]:
merged2

,Not_the_same,Age,Key_is_notCommon,City
0,frank,11,frank,Zaandam
1,sushi,15,sushi,Amstersdam
2,julie,14,julie,Rotterdam


# So that worked but now we have an extra column we don't need. We can drop it, but better to just name all the keys the same, and never anything else the same as the key

# Now about excel spreadsheets. If we have a key for all the sheets, we can write a function to simply read each sheet as a csv, then merge them


But we need to make sure they key columns actually match!
Let's see an example when they do not by adding a row to df1

In [15]:
df1

,Name,Age
0,frank,11
1,sushi,15
2,julie,14
3,amy,89


In [16]:
df1.loc[len(df1.index)] = ['amy', 89]
df1

,Name,Age
0,frank,11
1,sushi,15
2,julie,14
3,amy,89
4,amy,89


df2

In [17]:

merged5 = df1.merge(df2, left_on='Name', right_on='Name')
merged5

,Name,Age,City
0,frank,11,Zaandam
1,sushi,15,Amstersdam
2,julie,14,Rotterdam


# Where did poor Amy go? She was lost because we left the merge pattern on default.
Let's try again

In [18]:
merged5 = df1.merge(df2, how= 'outer',left_on='Name', right_on='Name')
merged5

,Name,Age,City
0,amy,89,NaN
1,amy,89,NaN
2,frank,11,Zaandam
3,julie,14,Rotterdam
4,sushi,15,Amstersdam


# This is just the basic level you need to know to start attaching more data once I am gone. I will build some functions to automate you tabular dta attachment this weeend.

# part #2: dealing with Excel

In [19]:
# Read Excel file with multiple sheets
xls = pd.ExcelFile("sample_synthetic_data/example_excel.xlsx")
# Get the list of sheet names
sheet_names = xls.sheet_names
# Print the sheet names
print(sheet_names)


['sheet1', 'sheet2', 'Sheet3row']


In [20]:
excel_file = pd.read_excel("sample_synthetic_data/example_excel.xlsx", sheet_name= sheet_names )

In [21]:
print(type(excel_file))


<class 'dict'>


So now we made a dictionary of the excel file. Let's access all parts of it

In [22]:
len(sheet_names)

3

In [23]:
sheet1 = excel_file[sheet_names[0]]
sheet1

,row_key,a,b,c
0,PatientA,alpha,bow,wow
1,PatientB,b eta,bow,ijn de bauw
2,PatientC,alpha,bow,bowwow
3,PatientD,b eta,bow,how
4,PatientE,theta,bow,now


In [24]:
sheet2 = excel_file[sheet_names[1]]
sheet3 = excel_file[sheet_names[1]]


In [25]:
sumsheet = sheet1.merge(sheet2, on='row_key')
sumsheet

,row_key,a,b,c,blank,na,blanky,d,e,f
0,PatientA,alpha,bow,wow,NaN,NaN,NaN,do,shcmu,do
1,PatientB,b eta,bow,ijn de bauw,NaN,NaN,NaN,do,do,schmu
2,PatientC,alpha,bow,bowwow,NaN,NaN,NaN,do,do,do
3,PatientD,b eta,bow,how,NaN,NaN,NaN,it,it,it
4,PatientE,theta,bow,now,NaN,NaN,NaN,now,now,now


In [26]:
sheet3

,row_key,blank,na,blanky,d,e,f
0,PatientA,NaN,NaN,NaN,do,shcmu,do
1,PatientB,NaN,NaN,NaN,do,do,schmu
2,PatientC,NaN,NaN,NaN,do,do,do
3,PatientD,NaN,NaN,NaN,it,it,it
4,PatientE,NaN,NaN,NaN,now,now,now


In [27]:
sumsheet = sumsheet.merge(sheet3, on='row_key')

In [28]:
sumsheet

,row_key,a,b,c,blank_x,na_x,blanky_x,d_x,e_x,f_x,blank_y,na_y,blanky_y,d_y,e_y,f_y
0,PatientA,alpha,bow,wow,NaN,NaN,NaN,do,shcmu,do,NaN,NaN,NaN,do,shcmu,do
1,PatientB,b eta,bow,ijn de bauw,NaN,NaN,NaN,do,do,schmu,NaN,NaN,NaN,do,do,schmu
2,PatientC,alpha,bow,bowwow,NaN,NaN,NaN,do,do,do,NaN,NaN,NaN,do,do,do
3,PatientD,b eta,bow,how,NaN,NaN,NaN,it,it,it,NaN,NaN,NaN,it,it,it
4,PatientE,theta,bow,now,NaN,NaN,NaN,now,now,now,NaN,NaN,NaN,now,now,now


# SO what was the bad idea here...we merged blank columsn with nothing in them...could we have dropped our empty columns beforehand?

In [29]:
sumsheet.dropna(how='all', axis=1, inplace=True)

In [30]:
sumsheet

,row_key,a,b,c,d_x,e_x,f_x,d_y,e_y,f_y
0,PatientA,alpha,bow,wow,do,shcmu,do,do,shcmu,do
1,PatientB,b eta,bow,ijn de bauw,do,do,schmu,do,do,schmu
2,PatientC,alpha,bow,bowwow,do,do,do,do,do,do
3,PatientD,b eta,bow,how,it,it,it,it,it,it
4,PatientE,theta,bow,now,now,now,now,now,now,now


In [31]:
# So now, let's put this all togehter in a function:

def csv_my_excel(excel_book_name, keyname):
    # Read Excel file with multiple sheets
    xls = pd.ExcelFile(excel_book_name)
    # Get the list of sheet names
    sheet_names = xls.sheet_names
    sheet_number= len(sheet_names)
    excel_file = pd.read_excel(excel_book_name, sheet_name= sheet_names)
    sheet_list = []
    for sheet_name in sheet_names:
        sheet = excel_file[sheet_name]
        sheet_list.append(sheet)
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=keyname,
                                            how='outer'), sheet_list)
    df_merged.dropna(how='all', axis=1, inplace=True)
    return df_merged
    

In [32]:
csv_my_excel("sample_synthetic_data/example_excel.xlsx", "row_key")

,row_key,a,b,c,d,e,f,fg,h
0,PatientA,alpha,bow,wow,do,shcmu,do,4,2
1,PatientB,b eta,bow,ijn de bauw,do,do,schmu,5,4
2,PatientC,alpha,bow,bowwow,do,do,do,7,3
3,PatientD,b eta,bow,how,it,it,it,8,3
4,PatientE,theta,bow,now,now,now,now,9,3


let's call it from our  budding lirbary

In [33]:
foundation.csv_my_excel("sample_synthetic_data/example_excel.xlsx", "row_key")

,row_key,a,b,c,d,e,f,fg,h
0,PatientA,alpha,bow,wow,do,shcmu,do,4,2
1,PatientB,b eta,bow,ijn de bauw,do,do,schmu,5,4
2,PatientC,alpha,bow,bowwow,do,do,do,7,3
3,PatientD,b eta,bow,how,it,it,it,8,3
4,PatientE,theta,bow,now,now,now,now,9,3
